# **Import and unzip the dataset**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#unzip the downloaded dataset
!unzip '/content/drive/MyDrive/damage vehicle.zip'

Archive:  /content/drive/MyDrive/damage vehicle.zip
   creating: damage vehicle/
   creating: damage vehicle/body/
   creating: damage vehicle/body/training/
   creating: damage vehicle/body/training/00-front/
  inflating: damage vehicle/body/training/00-front/0001.jpeg  
  inflating: damage vehicle/body/training/00-front/0002.JPEG  
  inflating: damage vehicle/body/training/00-front/0003.JPEG  
  inflating: damage vehicle/body/training/00-front/0004.JPEG  
  inflating: damage vehicle/body/training/00-front/0005.JPEG  
  inflating: damage vehicle/body/training/00-front/0006.JPEG  
  inflating: damage vehicle/body/training/00-front/0007.JPEG  
  inflating: damage vehicle/body/training/00-front/0008.jpeg  
  inflating: damage vehicle/body/training/00-front/0009.JPEG  
  inflating: damage vehicle/body/training/00-front/0010.JPEG  
  inflating: damage vehicle/body/training/00-front/0011.JPEG  
  inflating: damage vehicle/body/training/00-front/0012.jpeg  
  inflating: damage vehicle/body/t

# **Image Preprocessing**

**1. Import The ImageDataGenerator Library**

In [3]:
# Import required lib
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**2. Configure ImageDataGenerator Class**

In [4]:
#Creating augmentation on training variable
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

In [5]:
# Creating augmentation on testing variable

test_datagen = ImageDataGenerator(rescale=1./255)

**3. Apply ImageDataGenerator Functionality To Trainset And Testset**

In [6]:
# Passing training data to train variable for body

xtrain = train_datagen.flow_from_directory('/content/damage vehicle/body/training',
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size=10)

Found 979 images belonging to 3 classes.


In [7]:
# Passing testing data to test variable for body

xtest = test_datagen.flow_from_directory('/content/damage vehicle/body/validation',
                                         target_size=(224,224),
                                         class_mode='categorical',
                                         batch_size=10)

Found 171 images belonging to 3 classes.


In [8]:
# Passing training data to train variable for level

x_train = train_datagen.flow_from_directory('/content/damage vehicle/level/training',
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size=10)

Found 979 images belonging to 3 classes.


In [9]:
# Passing testing data to test variable for level

x_test = test_datagen.flow_from_directory('/content/damage vehicle/level/validation',
                                         target_size=(224,224),
                                         class_mode='categorical',
                                         batch_size=10)

Found 171 images belonging to 3 classes.


# **Model Building**

# **For Body**

**1. Importing The Model Building Libraries**

In [10]:
#Import the library
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [12]:
IMAGE_SIZE = [224, 224]

train_path = '/content/damage vehicle/body/training'
valid_path = '/content/damage vehicle/body/validation'

In [13]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


**3. Adding Flatten Layer**

In [14]:
for layer in vgg16.layers:
    layer.trainable = False

In [15]:
folders = glob('/content/damage vehicle/body/training/*')

In [16]:
folders

['/content/damage vehicle/body/training/00-front',
 '/content/damage vehicle/body/training/01-rear',
 '/content/damage vehicle/body/training/02-side']

In [17]:
x = Flatten()(vgg16.output)

In [18]:
len(folders)

3

**4. Adding Output Layer**

In [19]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [20]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**6. Configure The Learning Process**

In [22]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**7. Train The Model**

In [23]:
r = model.fit_generator(
  xtrain,
  validation_data=xtest,
  epochs=25,
  steps_per_epoch=len(xtrain),
  validation_steps=len(xtest)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 23s 146ms/step - loss: 1.2077 - accuracy: 0.5465 - val_loss: 1.2900 - val_accuracy: 0.5439
Epoch 2/25
98/98 [==============================] - 13s 128ms/step - loss: 0.8364 - accuracy: 0.7028 - val_loss: 0.8665 - val_accuracy: 0.6784
Epoch 3/25
98/98 [==============================] - 13s 128ms/step - loss: 0.5293 - accuracy: 0.7998 - val_loss: 1.3260 - val_accuracy: 0.5439
Epoch 4/25
98/98 [==============================] - 12s 127ms/step - loss: 0.3978 - accuracy: 0.8611 - val_loss: 0.9842 - val_accuracy: 0.6491
Epoch 5/25
98/98 [==============================] - 12s 127ms/step - loss: 0.2783 - accuracy: 0.9030 - val_loss: 0.9397 - val_accuracy: 0.6491
Epoch 6/25
98/98 [==============================] - 13s 128ms/step - loss: 0.2690 - accuracy: 0.9070 - val_loss: 0.9892 - val_accuracy: 0.6901
Epoch 7/25
98/98 [==============================] - 12s 127ms/step - loss: 0.1788 - accuracy: 0.9448 - val_loss: 1.0052 - val_accuracy: 0.7018

**8. Save The Model**

In [24]:
from tensorflow.keras.models import load_model

model.save('/content/damage vehicle/Model/body.h5')

**9. Test The Model**

In [25]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [26]:
model = load_model('/content/damage vehicle/Model/body.h5')

In [27]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [28]:
import numpy as np

In [29]:
data = "/content/damage vehicle/body/training/00-front/0002.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 0s 148ms/step
front


# **Model Building**

# **For Level**

**1. Importing The Model Building Libraries**

In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [31]:
IMAGE_SIZE = [224, 224]

train_path = '/content/damage vehicle/level/training'
valid_path = '/content/damage vehicle/level/validation'

In [32]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

**3. Adding Flatten Layer**

In [33]:
for layer in vgg16.layers:
    layer.trainable = False

In [34]:
folders = glob('/content/damage vehicle/level/training/*')

In [35]:
folders

['/content/damage vehicle/level/training/03-severe',
 '/content/damage vehicle/level/training/02-moderate',
 '/content/damage vehicle/level/training/01-minor']

In [36]:
x = Flatten()(vgg16.output)

In [37]:
len(folders)

3

**4. Adding Output Layer**

In [38]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [39]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [40]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

**6. Configure The Learning Process**

In [41]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**7. Train The Model**

In [42]:
r = model.fit_generator(
  x_train,
  validation_data=x_test,
  epochs=25,
  steps_per_epoch=len(x_train),
  validation_steps=len(x_test)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 14s 133ms/step - loss: 1.1629 - accuracy: 0.5495 - val_loss: 1.1559 - val_accuracy: 0.6199
Epoch 2/25
98/98 [==============================] - 13s 130ms/step - loss: 0.7157 - accuracy: 0.7089 - val_loss: 0.9643 - val_accuracy: 0.6491
Epoch 3/25
98/98 [==============================] - 13s 130ms/step - loss: 0.4978 - accuracy: 0.8161 - val_loss: 1.5663 - val_accuracy: 0.5322
Epoch 4/25
98/98 [==============================] - 13s 128ms/step - loss: 0.5277 - accuracy: 0.7865 - val_loss: 1.6003 - val_accuracy: 0.5088
Epoch 5/25
98/98 [==============================] - 13s 128ms/step - loss: 0.3763 - accuracy: 0.8468 - val_loss: 1.1925 - val_accuracy: 0.6082
Epoch 6/25
98/98 [==============================] - 13s 128ms/step - loss: 0.2445 - accuracy: 0.9203 - val_loss: 1.0354 - val_accuracy: 0.6374
Epoch 7/25
98/98 [==============================] - 13s 128ms/step - loss: 0.1902 - accuracy: 0.9346 - val_loss: 1.2155 - val_accuracy: 0.6550

**8. Save The Model**

In [43]:
from tensorflow.keras.models import load_model

model.save('/content/damage vehicle/Model/level.h5')

**9. Test The Model**

In [44]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [45]:
model = load_model('/content/damage vehicle/Model/level.h5')

In [46]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["minor","moderate","severe"]
  preds = label[np.argmax(prediction)]
  return preds

In [47]:
import numpy as np

In [48]:
data = "/content/damage vehicle/level/validation/01-minor/0005.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 0s 142ms/step
minor
